In [44]:
import numpy as np
import derivatives as der

In [39]:
collision = np.load("T100TTFT-C.npz", allow_pickle=True)
C = collision['C']
p = collision['p']

data = np.load("T100TTFT.npz", allow_pickle=True)
settings = data['settings'].item()
raw = data['raw'].item()
nu3D = raw['nu3D']
nubar3D = raw['nubar3D']

T = settings['T']
dm2 = settings['delta m-squared']
theta = np.arcsin(np.sqrt(settings['sin^2 (2theta)']))/2
N = settings['N'] + 1
E = np.linspace(0,settings['eps_max'], N) * T
time = raw['time']

In [41]:
H = np.zeros((len(time), len(time)-1, 4))

In [53]:
def H(i,j,k):
    vacuum = der.vacuum(nu3D[i,j,:], E[j+1], dm2, theta)
    Vvv = der.Vvv_function(nu3D[i,:,:], E[1:]) - der.Vvv_function(nubar3D[i,:,:], E[1:])
    VTbar = der.VT_barfunction(nu3D[i,:,:], nubar3D[i,:,:], E[1:], T)
    
    return nu3D[i,j,0]* (vacuum + der.crossproduct(nu3D[i,j,1:], Vvv) + der.crossproduct(nu3D[i,j,1:], VTbar))

In [54]:
def Hbar(i,j,k):
    vacuum = der.vacuum(nubar3D[i,j,:], E[j+1], dm2, theta)
    Vvv = der.Vvv_function(nu3D[i,:,:], E[1:]) - der.Vvv_function(nubar3D[i,:,:], E[1:])
    VTbar = der.VT_barfunction(nu3D[i,:,:], nubar3D[i,:,:], E[1:], T)
    
    return nubar3D[i,j,0]* (-vacuum + der.crossproduct(nubar3D[i,j,1:], Vvv) + der.crossproduct(nubar3D[i,j,1:], VTbar))